In [96]:
from sklearn import preprocessing

from dataset import *
from models.base_model import *
from models.neural_net import *
from models.sampled_net import *
from viewer.visualisator import *
from logic import *
import dataset
import models.base_model
import models.neural_net
import models.sampled_net
import viewer.visualisator
import logic

import importlib
from sklearn.pipeline import Pipeline
from swimnetworks import Dense, Linear

importlib.reload(dataset)
importlib.reload(models.base_model)
importlib.reload(models.neural_net)
importlib.reload(models.sampled_net)
importlib.reload(viewer.visualisator)
importlib.reload(logic)

<module 'logic' from 'c:\\Users\\daydo\\OneDrive\\Dokumente\\sampled-networks\\logic.py'>

In [97]:
xd = 2  # Input space dimensions
yd = 4  # Output dimension
num_samples = 1000  # Number of data points
epochs = 100  # Number of training epochs
l = [200]  # Number of neurons in each layer


dataset = Dataset()
dataset.create_dataset_Barron(xd, num_samples)
dataset.scale(preprocessing.MinMaxScaler())
dataset.split_train_test(0.2)

In [98]:
steps = [
    (
        "dense",
        Dense(
            layer_width=20, activation="relu", parameter_sampler="relu", random_seed=42
        ),
    ),
    ("linear", Linear(regularization_scale=1e-10)),
]
model = Pipeline(steps)

model.fit(dataset.X_train, dataset.y_train)

model_s = SampledNet()
model_s.weights = [
    model.get_params()["steps"][0][1].weights,
    model.get_params()["steps"][1][1].weights,
]
model_s.biases = [
    -model.get_params()["steps"][0][1].biases,
    -model.get_params()["steps"][1][1].biases,
]

ysampled = model_s.predict(dataset.X_train)
print(loss_mse(ysampled, dataset.y_train))

yswim = model.predict(dataset.X_train)
print(loss_mse(yswim, dataset.y_train))

# model.transform(dataset.X_test)

0.0035470236094513963
0.0035470236094513963


In [99]:
compute_weights_biases_layer1(dataset.X_train, dataset.y_train, model_s.weights, model_s.biases)

Layer1 sampling:   0%|          | 0/20 [00:00<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (800,) (20,) 